In [1]:
%env CUDA_VISIBLE_DEVICES=0 TOKENIZERS_PARALLELISM=true

env: CUDA_VISIBLE_DEVICES=0 TOKENIZERS_PARALLELISM=true


### Task 1

In [2]:
from task1.train import train

In [3]:
train(with_amp=False, scaler_type=None)

Loss: 0.6165 Accuracy: 92.918: 100%|██████████| 40/40 [00:39<00:00,  1.02it/s] 


In [4]:
train(with_amp=True, scaler_type="static")

Loss: 0.5857 Accuracy: 98.6872: 100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


In [5]:
train(with_amp=True, scaler_type="dynamic")

Loss: 0.5847 Accuracy: 98.7843: 100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


### Task 2

In [2]:
from task2.run_epoch import run_epoch, DataMode
from task2.dataset import get_texts

In [4]:
data_path = "task2/wikitext-103-raw-v1/train-00000-of-00002.txt"
texts = list(get_texts(data_path, max_lines=300000))

In [4]:
run_epoch(DataMode.BRAIN, texts)

100%|██████████| 18751/18751 [31:05<00:00, 10.05it/s]


,pad_token_ratio,time
count,18746.000000,18746.000000
mean,0.844768,0.099114
std,0.036115,0.001630
min,0.679980,0.032973
25%,0.821191,0.098730
50%,0.846289,0.098950
75%,0.869824,0.099207
max,0.966797,0.126127


In [5]:
run_epoch(DataMode.BIG_BRAIN, texts)

100%|██████████| 18751/18751 [14:33<00:00, 21.47it/s]


,pad_token_ratio,time
count,18746.000000,18746.000000
mean,0.650151,0.046283
std,0.076858,0.012042
min,0.000000,0.015003
25%,0.599435,0.037296
50%,0.653017,0.043890
75%,0.704913,0.052716
max,0.880918,0.110691


In [6]:
run_epoch(DataMode.ULTRA_BIG_BRAIN, texts)

100%|██████████| 18751/18751 [14:37<00:00, 21.36it/s]


,pad_token_ratio,time
count,18746.000000,18746.000000
mean,0.650458,0.046509
std,0.077305,0.012095
min,0.000000,0.012021
25%,0.599560,0.037486
50%,0.653737,0.044064
75%,0.704827,0.052992
max,0.895508,0.108156


In [7]:
run_epoch(DataMode.ULTRA_BIG_BRAIN, texts, k=5)

100%|██████████| 18817/18817 [06:34<00:00, 47.74it/s]


,pad_token_ratio,time
count,18812.000000,18812.000000
mean,0.068673,0.020735
std,0.071873,0.013861
min,0.000000,0.006112
25%,0.012083,0.007663
50%,0.022727,0.018079
75%,0.143750,0.028911
max,0.262500,0.099670


In [5]:
run_epoch(DataMode.ULTRA_DUPER_BIG_BRAIN, texts)

100%|██████████| 300001/300001 [01:05<00:00, 4596.88it/s]


,pad_token_ratio,time
count,5714.0,5714.000000
mean,0.0,0.160782
std,0.0,0.004858
min,0.0,0.084369
25%,0.0,0.158971
50%,0.0,0.161484
75%,0.0,0.163772
max,0.0,0.219913


### Task 3

In [1]:
from task3.run_epoch import profile_vit

In [3]:
own_profiler_summary = profile_vit("own")

Dataset already extracted
Train Data: 4322
Val Data: 1081


Train:   0%|          | 0/4 [00:00<?, ?it/s]

Train:  75%|███████▌  | 3/4 [01:45<00:35, 35.22s/it]


In [6]:
own_profiler_summary.sort_values(by="duration (ms)", ascending=False).head(30)

duration (ms)
function module                                                
backward transformer: Transformer                     50.510484
forward  transformer: Transformer                     33.093271
backward transformer.layers.5.0: Attention             2.764512
         transformer.layers.4.0: Attention             2.761995
         transformer.layers.0.0: Attention             2.761248
         transformer.layers.6.0: Attention             2.757941
         transformer.layers.7.0: Attention             2.756245
         transformer.layers.8.0: Attention             2.756000
         transformer.layers.2.0: Attention             2.756000
         transformer.layers.1.0: Attention             2.753973
         transformer.layers.3.0: Attention             2.751563
         transformer.layers.9.0: Attention             2.750667
         transformer.layers.10.0: Attention            2.749995
         transformer.layers.11.0: Attention            2.749749
forward  transformer.layers.8.0: Attention             1.871595
         to_patch_embedding: Sequential                1.801291
         transformer.layers.4.0: Attention             1.550677
backward transformer.layers.11.1: FeedForward          1.547936
forward  transformer.layers.2.0: Attention             1.526933
         transformer.layers.6.0: Attention             1.515989
backward transformer.layers.11.1.net: Sequential       1.513237
forward  transformer.layers.3.0: Attention             1.503360
         transformer.layers.10.0: Attention            1.481440
         transformer.layers.1.0: Attention             1.464384
         transformer.layers.11.0: Attention            1.436448
backward transformer.layers.0.1: FeedForward           1.430923
forward  transformer.layers.9.0: Attention             1.427381
backward transformer.layers.0.1.net: Sequential        1.425675
forward  transformer.layers.5.0: Attention             1.424693
         transformer.layers.7.0: Attention             1.422709

In [4]:
torch_profiler_summary = profile_vit("torch")

Dataset already extracted
Train Data: 4322
Val Data: 1081


Train:   0%|          | 0/17 [00:00<?, ?it/s]

Train:  18%|█▊        | 3/17 [02:34<11:59, 51.43s/it]


In [5]:
print(torch_profiler_summary.table(sort_by="cuda_time_total", row_limit=30))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               aten::mm         0.01%      10.058ms         0.02%      15.930ms      29.016us     137.431ms        35.09%     137.431ms     250.329us           549  
void cutlass::Kernel<cutlass_80_simt_sgemm_32x128_8x...         0.00%       0.000us         0.00%       0.000us       0.000us     113.860ms        29.07%     113.860ms     443.037us           257  
         

В целом похоже на то, что совпадает - но в torch-профилере есть детализация поглубже и в нем операции объединяются по типу, а в моем по модулю.
Но как будто бы более-менее похоже, линейный слой в атеншене - 250 наносекунд :)